## LangGraph Open Deep Research - Supervisor-Researcher Architecture

In this notebook, we'll explore the **supervisor-researcher delegation architecture** for conducting deep research with LangGraph.

You can visit this repository to see the original application: [Open Deep Research](https://github.com/langchain-ai/open_deep_research)

Let's jump in!

## What We're Building

This implementation uses a **hierarchical delegation pattern** where:

1. **User Clarification** - Optionally asks clarifying questions to understand the research scope
2. **Research Brief Generation** - Transforms user messages into a structured research brief
3. **Supervisor** - A lead researcher that analyzes the brief and delegates research tasks
4. **Parallel Researchers** - Multiple sub-agents that conduct focused research simultaneously
5. **Research Compression** - Each researcher synthesizes their findings
6. **Final Report** - All findings are combined into a comprehensive report

![Architecture Diagram](https://i.imgur.com/Q8HEZn0.png)

This differs from a section-based approach by allowing dynamic task decomposition based on the research question, rather than predefined sections.

---

# 🤝 Breakout Room #1
## Deep Research Foundations

In this breakout room, we'll understand the architecture and components of the Open Deep Research system.

## Task 1: Dependencies

You'll need API keys for Anthropic (for the LLM) and Tavily (for web search). We'll configure the system to use Anthropic's Claude Sonnet 4 exclusively.

In [1]:
import os
import getpass

os.environ["ANTHROPIC_API_KEY"] = getpass.getpass("Enter your Anthropic API key: ")
os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter your Tavily API key: ")

Enter your Anthropic API key:  ········
Enter your Tavily API key:  ········


## Task 2: State Definitions

The state structure is hierarchical with three levels:

### Agent State (Top Level)
Contains the overall conversation messages, research brief, accumulated notes, and final report.

### Supervisor State (Middle Level)
Manages the research supervisor's messages, research iterations, and coordinating parallel researchers.

### Researcher State (Bottom Level)
Each individual researcher has their own message history, tool call iterations, and research findings.

We also have structured outputs for tool calling:
- **ConductResearch** - Tool for supervisor to delegate research to a sub-agent
- **ResearchComplete** - Tool to signal research phase is done
- **ClarifyWithUser** - Structured output for asking clarifying questions
- **ResearchQuestion** - Structured output for the research brief

Let's import these from our library: [`open_deep_library/state.py`](open_deep_library/state.py)

In [2]:
# Import state definitions from the library
from open_deep_library.state import (
    # Main workflow states
    AgentState,           # Lines 65-72: Top-level agent state with messages, research_brief, notes, final_report
    AgentInputState,      # Lines 62-63: Input state is just messages
    
    # Supervisor states
    SupervisorState,      # Lines 74-81: Supervisor manages research delegation and iterations
    
    # Researcher states
    ResearcherState,      # Lines 83-90: Individual researcher with messages and tool iterations
    ResearcherOutputState, # Lines 92-96: Output from researcher (compressed research + raw notes)
    
    # Structured outputs for tool calling
    ConductResearch,      # Lines 15-19: Tool for delegating research to sub-agents
    ResearchComplete,     # Lines 21-22: Tool to signal research completion
    ClarifyWithUser,      # Lines 30-41: Structured output for user clarification
    ResearchQuestion,     # Lines 43-48: Structured output for research brief
)

## Task 3: Utility Functions and Tools

The system uses several key utilities:

### Search Tools
- **tavily_search** - Async web search with automatic summarization to stay within token limits
- Supports Anthropic native web search and Tavily API

### Reflection Tools
- **think_tool** - Allows researchers to reflect on their progress and plan next steps (ReAct pattern)

### Helper Utilities
- **get_all_tools** - Assembles the complete toolkit (search + MCP + reflection)
- **get_today_str** - Provides current date context for research
- Token limit handling utilities for graceful degradation

These are defined in [`open_deep_library/utils.py`](open_deep_library/utils.py)

In [3]:
# Import utility functions and tools from the library
from open_deep_library.utils import (
    # Search tool - Lines 43-136: Tavily search with automatic summarization
    tavily_search,
    
    # Reflection tool - Lines 219-244: Strategic thinking tool for ReAct pattern
    think_tool,
    
    # Tool assembly - Lines 569-597: Get all configured tools
    get_all_tools,
    
    # Date utility - Lines 872-879: Get formatted current date
    get_today_str,
    
    # Supporting utilities for error handling
    get_api_key_for_model,          # Lines 892-914: Get API keys from config or env
    is_token_limit_exceeded,         # Lines 665-701: Detect token limit errors
    get_model_token_limit,           # Lines 831-846: Look up model's token limit
    remove_up_to_last_ai_message,    # Lines 848-866: Truncate messages for retry
    anthropic_websearch_called,      # Lines 607-637: Detect Anthropic native search usage
    openai_websearch_called,         # Lines 639-658: Detect OpenAI native search usage
    get_notes_from_tool_calls,       # Lines 599-601: Extract notes from tool messages
)

## Task 4: Configuration System

The configuration system controls:

### Research Behavior
- **allow_clarification** - Whether to ask clarifying questions before research
- **max_concurrent_research_units** - How many parallel researchers can run (default: 5)
- **max_researcher_iterations** - How many times supervisor can delegate research (default: 6)
- **max_react_tool_calls** - Tool call limit per researcher (default: 10)

### Model Configuration
- **research_model** - Model for research and supervision (we'll use Anthropic)
- **compression_model** - Model for synthesizing findings
- **final_report_model** - Model for writing the final report
- **summarization_model** - Model for summarizing web search results

### Search Configuration
- **search_api** - Which search API to use (ANTHROPIC, TAVILY, or NONE)
- **max_content_length** - Character limit before summarization

Defined in [`open_deep_library/configuration.py`](open_deep_library/configuration.py)

In [4]:
# Import configuration from the library
from open_deep_library.configuration import (
    Configuration,    # Lines 38-247: Main configuration class with all settings
    SearchAPI,        # Lines 11-17: Enum for search API options (ANTHROPIC, TAVILY, NONE)
)

## Task 5: Prompt Templates

The system uses carefully engineered prompts for each phase:

### Phase 1: Clarification
**clarify_with_user_instructions** - Analyzes if the research scope is clear or needs clarification

### Phase 2: Research Brief
**transform_messages_into_research_topic_prompt** - Converts user messages into a detailed research brief

### Phase 3: Supervisor
**lead_researcher_prompt** - System prompt for the supervisor that manages delegation strategy

### Phase 4: Researcher
**research_system_prompt** - System prompt for individual researchers conducting focused research

### Phase 5: Compression
**compress_research_system_prompt** - Prompt for synthesizing research findings without losing information

### Phase 6: Final Report
**final_report_generation_prompt** - Comprehensive prompt for writing the final report

All prompts are defined in [`open_deep_library/prompts.py`](open_deep_library/prompts.py)

In [5]:
# Import prompt templates from the library
from open_deep_library.prompts import (
    clarify_with_user_instructions,                    # Lines 3-41: Ask clarifying questions
    transform_messages_into_research_topic_prompt,     # Lines 44-77: Generate research brief
    lead_researcher_prompt,                            # Lines 79-136: Supervisor system prompt
    research_system_prompt,                            # Lines 138-183: Researcher system prompt
    compress_research_system_prompt,                   # Lines 186-222: Research compression prompt
    final_report_generation_prompt,                    # Lines 228-308: Final report generation
)

## ❓ Question #1:

Explain the interrelationships between the three states (Agent, Supervisor, Researcher). Why don't we just make a single huge state?

##### Answer:
- all agents only need context for their specific roles to prevent context bloating/context rotting
- isolate state so that each agent can act on the state they actually need
- subagents can work in parallel once they have their own state
- huge state could work but there will be context bloating and not every agent needs the specifics of another agent's state
- research loop => subagents may find things that do not agree (so research supervisor will use the loop to clarify things and make the final report make sense)
- need summarization before report generation which is compression so that the important things fit in the report agent's context window (if it's too large, the report agent will fail)

## ❓ Question #2:

What are the advantages and disadvantages of importing these components instead of including them in the notebook?

##### Answer:
- advantages:
  - allows you to reuse code in other scripts/modules instead of just defining them inside a notebook (which makes them hard to import elsewhere)
  - good for reuse when building production systems

- disadvantages:
  - easier to inspect the code if they are defined inside a notebook
  - easier to redefine methods/functions later when experimenting inside a notebook
  - harder to experiment with the code (harder to see intermediate output without notebook support)

## 🏗️ Activity #1: Explore the Prompts

Open `open_deep_library/prompts.py` and examine one of the prompt templates in detail.

**Requirements:**
1. Choose one prompt template (clarify, brief, supervisor, researcher, compression, or final report)
2. Explain what the prompt is designed to accomplish
3. Identify 2-3 key techniques used in the prompt (e.g., structured output, role definition, examples)
4. Suggest one improvement you might make to the prompt

**YOUR CODE HERE** - Write your analysis in a markdown cell below

## Compress Research System Prompt

compress_research_system_prompt = """You are a research assistant that has conducted research on a topic by calling several tools and web searches. Your job is now to clean up the findings, but preserve all of the relevant statements and information that the researcher has gathered. For context, today's date is {date}.

<Task>
You need to clean up information gathered from tool calls and web searches in the existing messages.
All relevant information should be repeated and rewritten verbatim, but in a cleaner format.
The purpose of this step is just to remove any obviously irrelevant or duplicative information.
For example, if three sources all say "X", you could say "These three sources all stated X".
Only these fully comprehensive cleaned findings are going to be returned to the user, so it's crucial that you don't lose any information from the raw messages.
</Task>

<Guidelines>
1. Your output findings should be fully comprehensive and include ALL of the information and sources that the researcher has gathered from tool calls and web searches. It is expected that you repeat key information verbatim.
2. This report can be as long as necessary to return ALL of the information that the researcher has gathered.
3. In your report, you should return inline citations for each source that the researcher found.
4. You should include a "Sources" section at the end of the report that lists all of the sources the researcher found with corresponding citations, cited against statements in the report.
5. Make sure to include ALL of the sources that the researcher gathered in the report, and how they were used to answer the question!
6. It's really important not to lose any sources. A later LLM will be used to merge this report with others, so having all of the sources is critical.
</Guidelines>

<Output Format>
The report should be structured like this:
**List of Queries and Tool Calls Made**
**Fully Comprehensive Findings**
**List of All Relevant Sources (with citations in the report)**
</Output Format>

<Citation Rules>
- Assign each unique URL a single citation number in your text
- End with ### Sources that lists each source with corresponding numbers
- IMPORTANT: Number sources sequentially without gaps (1,2,3,4...) in the final list regardless of which sources you choose
- Example format:
  [1] Source Title: URL
  [2] Source Title: URL
</Citation Rules>

Critical Reminder: It is extremely important that any information that is even remotely relevant to the user's research topic is preserved verbatim (e.g. don't rewrite it, don't summarize it, don't paraphrase it).
"""

1) The prompt turns raw research outputs (tool calls and web search results) into a single, cleaned report that:
- Keeps all relevant information and does not drop or summarize away facts.
- Removes only obvious noise and duplication (e.g., merging “three sources said X” into one clear sentence).
- Preserves important statements verbatim so a later LLM can merge this report with others without losing content.
- Adds inline citations and a numbered Sources section so every source is traceable and reusable.

2)
- structured output -> the system prompt forces a specific report shape
- role and task framing -> it defines a clear role and puts in core instructions (focuses the model on cleanup and preservation instead of summarization)
-  explicit citation rules -> reminds repeatedly for citations and stresses preserving everything so no sources are dropped

3. You could add one line on conflicting information such as this (“If sources disagree, keep both statements and cite each source”)
- you don't want the model dropping sources due to conflict


---

# 🤝 Breakout Room #2
## Building & Running the Researcher

In this breakout room, we'll explore the node functions, build the graph, and run wellness research.

## Task 6: Node Functions - The Building Blocks

Now let's look at the node functions that make up our graph. We'll import them from the library and understand what each does.

### The Complete Research Workflow

The workflow consists of 8 key nodes organized into 3 subgraphs:

1. **Main Graph Nodes:**
   - `clarify_with_user` - Entry point that checks if clarification is needed
   - `write_research_brief` - Transforms user input into structured research brief
   - `final_report_generation` - Synthesizes all research into final report

2. **Supervisor Subgraph Nodes:**
   - `supervisor` - Lead researcher that plans and delegates
   - `supervisor_tools` - Executes supervisor's tool calls (delegation, reflection)

3. **Researcher Subgraph Nodes:**
   - `researcher` - Individual researcher conducting focused research
   - `researcher_tools` - Executes researcher's tool calls (search, reflection)
   - `compress_research` - Synthesizes researcher's findings

All nodes are defined in [`open_deep_library/deep_researcher.py`](open_deep_library/deep_researcher.py)

### Node 1: clarify_with_user

**Purpose:** Analyzes user messages and asks clarifying questions if the research scope is unclear.

**Key Steps:**
1. Check if clarification is enabled in configuration
2. Use structured output to analyze if clarification is needed
3. If needed, end with a clarifying question for the user
4. If not needed, proceed to research brief with verification message

**Implementation:** [`open_deep_library/deep_researcher.py` lines 60-115](open_deep_library/deep_researcher.py#L60-L115)

In [6]:
# Import the clarify_with_user node
from open_deep_library.deep_researcher import clarify_with_user

### Node 2: write_research_brief

**Purpose:** Transforms user messages into a structured research brief for the supervisor.

**Key Steps:**
1. Use structured output to generate detailed research brief from messages
2. Initialize supervisor with system prompt and research brief
3. Set up supervisor messages with proper context

**Why this matters:** A well-structured research brief helps the supervisor make better delegation decisions.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 118-175](open_deep_library/deep_researcher.py#L118-L175)

In [7]:
# Import the write_research_brief node
from open_deep_library.deep_researcher import write_research_brief

### Node 3: supervisor

**Purpose:** Lead research supervisor that plans research strategy and delegates to sub-researchers.

**Key Steps:**
1. Configure model with three tools:
   - `ConductResearch` - Delegate research to a sub-agent
   - `ResearchComplete` - Signal that research is done
   - `think_tool` - Strategic reflection before decisions
2. Generate response based on current context
3. Increment research iteration count
4. Proceed to tool execution

**Decision Making:** The supervisor uses `think_tool` to reflect before delegating research, ensuring thoughtful decomposition of the research question.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 178-223](open_deep_library/deep_researcher.py#L178-L223)

In [8]:
# Import the supervisor node (from supervisor subgraph)
from open_deep_library.deep_researcher import supervisor

### Node 4: supervisor_tools

**Purpose:** Executes the supervisor's tool calls, including strategic thinking and research delegation.

**Key Steps:**
1. Check exit conditions:
   - Exceeded maximum iterations
   - No tool calls made
   - `ResearchComplete` called
2. Process `think_tool` calls for strategic reflection
3. Execute `ConductResearch` calls in parallel:
   - Spawn researcher subgraphs for each delegation
   - Limit to `max_concurrent_research_units` (default: 5)
   - Gather all results asynchronously
4. Aggregate findings and return to supervisor

**Parallel Execution:** This is where the magic happens - multiple researchers work simultaneously on different aspects of the research question.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 225-349](open_deep_library/deep_researcher.py#L225-L349)

In [9]:
# Import the supervisor_tools node
from open_deep_library.deep_researcher import supervisor_tools

### Node 5: researcher

**Purpose:** Individual researcher that conducts focused research on a specific topic.

**Key Steps:**
1. Load all available tools (search, MCP, reflection)
2. Configure model with tools and researcher system prompt
3. Generate response with tool calls
4. Increment tool call iteration count

**ReAct Pattern:** Researchers use `think_tool` to reflect after each search, deciding whether to continue or provide their answer.

**Available Tools:**
- Search tools (Tavily or Anthropic native search)
- `think_tool` for strategic reflection
- `ResearchComplete` to signal completion
- MCP tools (if configured)

**Implementation:** [`open_deep_library/deep_researcher.py` lines 365-424](open_deep_library/deep_researcher.py#L365-L424)

In [10]:
# Import the researcher node (from researcher subgraph)
from open_deep_library.deep_researcher import researcher

### Node 6: researcher_tools

**Purpose:** Executes the researcher's tool calls, including searches and strategic reflection.

**Key Steps:**
1. Check early exit conditions (no tool calls, native search used)
2. Execute all tool calls in parallel:
   - Search tools fetch and summarize web content
   - `think_tool` records strategic reflections
   - MCP tools execute external integrations
3. Check late exit conditions:
   - Exceeded `max_react_tool_calls` (default: 10)
   - `ResearchComplete` called
4. Continue research loop or proceed to compression

**Error Handling:** Safely handles tool execution errors and continues with available results.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 435-509](open_deep_library/deep_researcher.py#L435-L509)

In [11]:
# Import the researcher_tools node
from open_deep_library.deep_researcher import researcher_tools

### Node 7: compress_research

**Purpose:** Compresses and synthesizes research findings into a concise, structured summary.

**Key Steps:**
1. Configure compression model
2. Add compression instruction to messages
3. Attempt compression with retry logic:
   - If token limit exceeded, remove older messages
   - Retry up to 3 times
4. Extract raw notes from tool and AI messages
5. Return compressed research and raw notes

**Why Compression?** Researchers may accumulate lots of tool outputs and reflections. Compression ensures:
- All important information is preserved
- Redundant information is deduplicated
- Content stays within token limits for the final report

**Token Limit Handling:** Gracefully handles token limit errors by progressively truncating messages.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 511-585](open_deep_library/deep_researcher.py#L511-L585)

In [12]:
# Import the compress_research node
from open_deep_library.deep_researcher import compress_research

### Node 8: final_report_generation

**Purpose:** Generates the final comprehensive research report from all collected findings.

**Key Steps:**
1. Extract all notes from completed research
2. Configure final report model
3. Attempt report generation with retry logic:
   - If token limit exceeded, truncate findings by 10%
   - Retry up to 3 times
4. Return final report or error message

**Token Limit Strategy:**
- First retry: Use model's token limit × 4 as character limit
- Subsequent retries: Reduce by 10% each time
- Graceful degradation with helpful error messages

**Report Quality:** The prompt guides the model to create well-structured reports with:
- Proper headings and sections
- Inline citations
- Comprehensive coverage of all findings
- Sources section at the end

**Implementation:** [`open_deep_library/deep_researcher.py` lines 607-697](open_deep_library/deep_researcher.py#L607-L697)

In [13]:
# Import the final_report_generation node
from open_deep_library.deep_researcher import final_report_generation

## Task 7: Graph Construction - Putting It All Together

The system is organized into three interconnected graphs:

### 1. Researcher Subgraph (Bottom Level)
Handles individual focused research on a specific topic:
```
START → researcher → researcher_tools → compress_research → END
               ↑            ↓
               └────────────┘ (loops until max iterations or ResearchComplete)
```

### 2. Supervisor Subgraph (Middle Level)
Manages research delegation and coordination:
```
START → supervisor → supervisor_tools → END
            ↑              ↓
            └──────────────┘ (loops until max iterations or ResearchComplete)
            
supervisor_tools spawns multiple researcher_subgraphs in parallel
```

### 3. Main Deep Researcher Graph (Top Level)
Orchestrates the complete research workflow:
```
START → clarify_with_user → write_research_brief → research_supervisor → final_report_generation → END
                 ↓                                       (supervisor_subgraph)
               (may end early if clarification needed)
```

Let's import the compiled graphs from the library.

In [14]:
# Import the pre-compiled graphs from the library
from open_deep_library.deep_researcher import (
    # Bottom level: Individual researcher workflow
    researcher_subgraph,    # Lines 588-605: researcher → researcher_tools → compress_research
    
    # Middle level: Supervisor coordination
    supervisor_subgraph,    # Lines 351-363: supervisor → supervisor_tools (spawns researchers)
    
    # Top level: Complete research workflow
    deep_researcher,        # Lines 699-719: Main graph with all phases
)

## Why This Architecture?

### Advantages of Supervisor-Researcher Delegation

1. **Dynamic Task Decomposition**
   - Unlike section-based approaches with predefined structure, the supervisor can break down research based on the actual question
   - Adapts to different types of research (comparisons, lists, deep dives, etc.)

2. **Parallel Execution**
   - Multiple researchers work simultaneously on different aspects
   - Much faster than sequential section processing
   - Configurable parallelism (1-20 concurrent researchers)

3. **ReAct Pattern for Quality**
   - Researchers use `think_tool` to reflect after each search
   - Prevents excessive searching and improves search quality
   - Natural stopping conditions based on information sufficiency

4. **Flexible Tool Integration**
   - Easy to add MCP tools for specialized research
   - Supports multiple search APIs (Anthropic, Tavily)
   - Each researcher can use different tool combinations

5. **Graceful Token Limit Handling**
   - Compression prevents token overflow
   - Progressive truncation in final report generation
   - Research can scale to arbitrary depths

### Trade-offs

- **Complexity:** More moving parts than section-based approach
- **Cost:** Parallel researchers use more tokens (but faster)
- **Unpredictability:** Research structure emerges dynamically

## Task 8: Running the Deep Researcher

Now let's see the system in action! We'll use it to research wellness strategies for improving sleep quality.

### Setup

We need to:
1. Set up the wellness research request
2. Configure the execution with Anthropic settings
3. Run the research workflow

In [15]:
# Set up the graph with Anthropic configuration
from IPython.display import Markdown, display
import uuid

# Note: deep_researcher is already compiled from the library
# For this demo, we'll use it directly without additional checkpointing
graph = deep_researcher

print("✓ Graph ready for execution")
print("  (Note: The graph is pre-compiled from the library)")

✓ Graph ready for execution
  (Note: The graph is pre-compiled from the library)


### Configuration for Anthropic

We'll configure the system to use:
- **Claude Sonnet 4** for all research, supervision, and report generation
- **Tavily** for web search (you can also use Anthropic's native search)
- **Moderate parallelism** (1 concurrent researcher for cost control)
- **Clarification enabled** (will ask if research scope is unclear)

In [16]:
# Configure for Anthropic with moderate settings
config = {
    "configurable": {
        # Model configuration - using Claude Sonnet 4 for everything
        "research_model": "anthropic:claude-sonnet-4-20250514",
        "research_model_max_tokens": 10000,
        
        "compression_model": "anthropic:claude-sonnet-4-20250514",
        "compression_model_max_tokens": 8192,
        
        "final_report_model": "anthropic:claude-sonnet-4-20250514",
        "final_report_model_max_tokens": 10000,
        
        "summarization_model": "anthropic:claude-sonnet-4-20250514",
        "summarization_model_max_tokens": 8192,
        
        # Research behavior
        "allow_clarification": True,
        "max_concurrent_research_units": 1,  # 1 parallel researcher
        "max_researcher_iterations": 2,      # Supervisor can delegate up to 2 times
        "max_react_tool_calls": 3,           # Each researcher can make up to 3 tool calls
        
        # Search configuration
        "search_api": "tavily",  # Using Tavily for web search
        "max_content_length": 50000,
        
        # Thread ID for this conversation
        "thread_id": str(uuid.uuid4())
    }
}

print("✓ Configuration ready")
print(f"  - Research Model: Claude Sonnet 4")
print(f"  - Max Concurrent Researchers: 1")
print(f"  - Max Iterations: 2")
print(f"  - Search API: Tavily")

✓ Configuration ready
  - Research Model: Claude Sonnet 4
  - Max Concurrent Researchers: 1
  - Max Iterations: 2
  - Search API: Tavily


### Execute the Wellness Research

Now let's run the research! We'll ask the system to research evidence-based strategies for improving sleep quality.

The workflow will:
1. **Clarify** - Check if the request is clear (may skip if obvious)
2. **Research Brief** - Transform our request into a structured brief
3. **Supervisor** - Plan research strategy and delegate to researchers
4. **Parallel Research** - Researchers gather information simultaneously
5. **Compression** - Each researcher synthesizes their findings
6. **Final Report** - All findings combined into comprehensive report

In [19]:
# Create our wellness research request
research_request = """
I want to improve my sleep quality. I currently:
- Go to bed at inconsistent times (10pm-1am)
- Use my phone in bed
- Often feel tired in the morning

Please research the best evidence-based strategies for improving sleep quality and create a comprehensive sleep improvement plan for me.
"""

# Execute the graph
async def run_research():
    """Run the research workflow and display results."""
    print("Starting research workflow...\n")
    
    async for event in graph.astream(
        {"messages": [{"role": "user", "content": research_request}]},
        config,
        stream_mode="updates"
    ):
        # Display each step
        for node_name, node_output in event.items():
            print(f"\n{'='*60}")
            print(f"Node: {node_name}")
            print(f"{'='*60}")
            
            if node_name == "clarify_with_user":
                if "messages" in node_output:
                    last_msg = node_output["messages"][-1]
                    print(f"\n{last_msg.content}")
            
            elif node_name == "write_research_brief":
                if "research_brief" in node_output:
                    print(f"\nResearch Brief Generated:")
                    print(f"{node_output['research_brief'][:500]}...")
            
            elif node_name == "supervisor":
                print(f"\nSupervisor planning research strategy...")
                if "supervisor_messages" in node_output:
                    last_msg = node_output["supervisor_messages"][-1]
                    if hasattr(last_msg, 'tool_calls') and last_msg.tool_calls:
                        print(f"Tool calls: {len(last_msg.tool_calls)}")
                        for tc in last_msg.tool_calls:
                            print(f"  - {tc['name']}")
            
            elif node_name == "supervisor_tools":
                print(f"\nExecuting supervisor's tool calls...")
                if "notes" in node_output:
                    print(f"Research notes collected: {len(node_output['notes'])}")
            
            elif node_name == "final_report_generation":
                if "final_report" in node_output:
                    print(f"\n" + "="*60)
                    print("FINAL REPORT GENERATED")
                    print("="*60 + "\n")
                    display(Markdown(node_output["final_report"]))
    
    print("\n" + "="*60)
    print("Research workflow completed!")
    print("="*60)

# Run the research
await run_research()

Starting research workflow...


Node: clarify_with_user

I have sufficient information to proceed with your sleep improvement research. I understand you want evidence-based strategies to address your current sleep challenges: inconsistent bedtime (10pm-1am), phone use in bed, and morning fatigue. I'll research comprehensive, science-backed sleep improvement strategies and create a personalized sleep plan based on your specific situation. I'll now begin the research process.

Node: write_research_brief

Research Brief Generated:
I want to improve my sleep quality based on evidence-based strategies. My current sleep challenges include: going to bed at inconsistent times (ranging from 10pm to 1am), using my phone in bed, and often feeling tired in the morning despite sleep. Please research comprehensive, science-backed sleep improvement strategies and create a personalized sleep plan that specifically addresses these issues. The research should cover: (1) optimal sleep timing and consiste


Node: research_supervisor

Node: final_report_generation

FINAL REPORT GENERATED



Error generating final report: Error code: 429 - {'type': 'error', 'error': {'type': 'rate_limit_error', 'message': "This request would exceed your organization's rate limit of 30,000 input tokens per minute (org: dfd07c34-45be-47ed-a2b1-05590ecb18ea, model: claude-sonnet-4-20250514). For details, refer to: https://docs.claude.com/en/api/rate-limits. You can see the response headers for current usage. Please reduce the prompt length or the maximum tokens requested, or try again later. You may also contact sales at https://www.anthropic.com/contact-sales to discuss your options for a rate limit increase."}, 'request_id': 'req_011CY19vUSFKUiqQbFNLwUaH'}


Research workflow completed!


## Task 9: Understanding the Output

Let's break down what happened:

### Phase 1: Clarification
The system checked if your request was clear. Since you provided specific details about your sleep issues, it likely proceeded without asking clarifying questions.

### Phase 2: Research Brief
Your request was transformed into a detailed research brief that guides the supervisor's delegation strategy.

### Phase 3: Supervisor Delegation
The supervisor analyzed the brief and decided how to break down the research:
- Used `think_tool` to plan strategy
- Called `ConductResearch` to delegate to researchers
- Each delegation specified a focused research topic (e.g., sleep hygiene, circadian rhythm, blue light effects)

### Phase 4: Parallel Research
Researchers worked on their assigned topics:
- Each researcher used web search tools to gather information
- Used `think_tool` to reflect after each search
- Decided when they had enough information
- Compressed their findings into clean summaries

### Phase 5: Final Report
All research findings were synthesized into a comprehensive sleep improvement plan with:
- Well-structured sections
- Evidence-based recommendations
- Practical action items
- Sources for further reading

## Task 10: Key Takeaways & Next Steps

### Architecture Benefits
1. **Dynamic Decomposition** - Research structure emerges from the question, not predefined
2. **Parallel Efficiency** - Multiple researchers work simultaneously
3. **ReAct Quality** - Strategic reflection improves search decisions
4. **Scalability** - Handles token limits gracefully through compression
5. **Flexibility** - Easy to add new tools and capabilities

### When to Use This Pattern
- **Complex research questions** that need multi-angle investigation
- **Comparison tasks** where parallel research on different topics is beneficial
- **Open-ended exploration** where structure should emerge dynamically
- **Time-sensitive research** where parallel execution speeds up results

### When to Use Section-Based Instead
- **Highly structured reports** with predefined format requirements
- **Template-based content** where sections are always the same
- **Sequential dependencies** where later sections depend on earlier ones
- **Budget constraints** where token efficiency is critical

### Extend the System
1. **Add MCP Tools** - Integrate specialized tools for your domain
2. **Custom Prompts** - Modify prompts for specific research types
3. **Different Models** - Try different Claude versions or mix models
4. **Persistence** - Use a real database for checkpointing instead of memory

### Learn More
- [LangGraph Documentation](https://langchain-ai.github.io/langgraph/)
- [Open Deep Research Repo](https://github.com/langchain-ai/open_deep_research)
- [Anthropic Claude Documentation](https://docs.anthropic.com/)
- [Tavily Search API](https://tavily.com/)

## ❓ Question #3:

What are the trade-offs of using parallel researchers vs. sequential research? When might you choose one approach over the other?

##### Answer:
- parallel -> different research tasks that are independent of each other (can be accomplished in parallel since there is no dependency)
- sequential -> tasks that depend on each other (report generator agent cannot generate reports until the research agent is done)

## ❓ Question #4:

How would you adapt this deep research architecture for a production wellness application? What additional components would you need?

##### Answer:
- you would need user preferences and also sort of memory
- you may want RAG to incorporate more health and wealthness data directly into your app (instead of just online sources)
- you may want the ability to re-iterate on research questions directly from the user before turning them into research briefs

## 🏗️ Activity #2: Custom Wellness Research

Using what you've learned, run a custom wellness research task.

**Requirements:**
1. Create a wellness-related research question (exercise, nutrition, stress, etc.)
2. Modify the configuration for your use case
3. Run the research and analyze the output
4. Document what worked well and what could be improved

**Experiment ideas:**
- Research exercise routines for specific conditions (bad knee, lower back pain)
- Compare different stress management techniques
- Investigate nutrition strategies for specific goals
- Explore meditation and mindfulness research

**YOUR CODE HERE**

In [21]:
# YOUR CODE HERE
from IPython.display import Markdown, display
import uuid

# Create your own wellness research request and run it

my_wellness_request = """
I have a hard time keeping a regular sleep schedule. I tend to be a night owl and no matter how hard I try, I naturally tend to feel tired a few hours later than most people.

Please research what the long-term health implications are of being a night owl. If you tend to get enough sleep, does it actually matter what time you go to bed, even if it's later than most people?
"""

# Optionally modify the config
my_config = {
    "configurable": {
        "research_model": "anthropic:claude-sonnet-4-20250514",
        "research_model_max_tokens": 10000,
        "compression_model": "anthropic:claude-sonnet-4-20250514",
        "compression_model_max_tokens": 8192,
        "final_report_model": "anthropic:claude-sonnet-4-20250514",
        "final_report_model_max_tokens": 10000,
        "summarization_model": "anthropic:claude-sonnet-4-20250514",
        "summarization_model_max_tokens": 8192,
        "allow_clarification": True,
        "max_concurrent_research_units": 1,
        "max_researcher_iterations": 2,
        "max_react_tool_calls": 3,
        "search_api": "tavily",
        "max_content_length": 50000,
        "thread_id": str(uuid.uuid4()),
    }
}

async def run_custom_research(research_request: str, config: dict):
    """Run the deep researcher graph and print/display key phases and final report."""
    print("Starting custom wellness research workflow...\n")

    async for event in graph.astream(
        {"messages": [{"role": "user", "content": research_request}]},
        config,
        stream_mode="updates",
    ):
        for node_name, node_output in event.items():
            print(f"\n{'='*60}")
            print(f"Node: {node_name}")
            print(f"{'='*60}")

            if node_name == "clarify_with_user":
                if "messages" in node_output:
                    last_msg = node_output["messages"][-1]
                    print(f"\n{last_msg.content}")

            elif node_name == "write_research_brief":
                if "research_brief" in node_output:
                    print("\nResearch Brief Generated:")
                    brief = node_output["research_brief"]
                    print(f"{brief[:500]}..." if len(brief) > 500 else brief)

            elif node_name == "supervisor":
                print("\nSupervisor planning research strategy...")
                if "supervisor_messages" in node_output:
                    last_msg = node_output["supervisor_messages"][-1]
                    if hasattr(last_msg, "tool_calls") and last_msg.tool_calls:
                        print(f"Tool calls: {len(last_msg.tool_calls)}")
                        for tc in last_msg.tool_calls:
                            print(f"  - {tc['name']}")

            elif node_name == "supervisor_tools":
                print("\nExecuting supervisor's tool calls...")
                if "notes" in node_output:
                    print(f"Research notes collected: {len(node_output['notes'])}")

            elif node_name == "final_report_generation":
                if "final_report" in node_output:
                    print("\n" + "=" * 60)
                    print("FINAL REPORT GENERATED")
                    print("=" * 60 + "\n")
                    display(Markdown(node_output["final_report"]))

    print("\n" + "=" * 60)
    print("Research workflow completed!")
    print("=" * 60)

In [22]:
# Run your research
await run_custom_research(my_wellness_request, my_config)

Starting custom wellness research workflow...


Node: clarify_with_user

I have sufficient information to proceed with your research request. I understand you're looking for information about the long-term health implications of being a night owl, specifically whether it matters what time you go to bed if you're still getting adequate sleep. I'll research the health effects of late sleep schedules, circadian rhythm impacts, and whether sleep timing affects health outcomes when sleep duration is adequate. I'll now begin the research process.

Node: write_research_brief

Research Brief Generated:
I have a hard time keeping a regular sleep schedule and tend to be a night owl, naturally feeling tired a few hours later than most people. I want to understand the long-term health implications of being a night owl and specifically whether it actually matters what time I go to bed if I'm still getting enough total sleep. Please research: (1) What are the documented long-term health effects and 


Node: research_supervisor

Node: final_report_generation

FINAL REPORT GENERATED



# The Health Implications of Being a Night Owl: A Comprehensive Analysis

Being a night owl, or having a late chronotype, involves more than simply preferring later bedtimes. Extensive research reveals that night owl sleep patterns carry significant long-term health implications that extend beyond just sleep duration, affecting multiple physiological systems even when adequate total sleep is maintained.

## Long-Term Health Risks Associated with Night Owl Chronotype

### Mortality and Cardiovascular Disease
Large-scale longitudinal studies demonstrate substantial health risks for night owls. The Finnish Twin Cohort Study, analyzing 24,000 twins over multiple years, found that night owls had a 9% increased risk of early death compared to morning people [1]. Even more concerning, the UK Biobank study following over 322,000 participants for 14 years revealed that night owls were 79% more likely to have poor overall cardiovascular health and faced a 16% higher risk of developing cardiovascular disease, including heart attacks and strokes [2].

This cardiovascular risk appears particularly pronounced in women, with the connection being strongest among female night owls. The study used the American Heart Association's Life's Essential 8 framework, measuring eight critical health factors including diet quality, physical activity, smoking status, sleep quality, weight management, cholesterol levels, blood sugar control, and blood pressure regulation [2].

### Mental Health Disorders
Mental health outcomes show similarly concerning patterns for night owls. Stanford Medicine research tracking nearly 75,000 adults over eight years found that night owls who maintained their natural late bedtime were 20% to 40% more likely to be diagnosed with mental health disorders, including depression and anxiety, compared to night owls who adopted earlier sleep schedules [3]. The study revealed that both morning larks and night owls who went to sleep late had higher rates of mental health disorders, with researchers recommending bedtimes no later than 1 a.m.

This aligns with the "mind after midnight" hypothesis, which suggests that neurological and physiological changes occurring late at night foster impulsivity, negative mood, impaired judgment, and risk-taking behavior. Research indicates that many harmful behaviors are more common at night, including suicidal thinking, violent crimes, alcohol and drug use, and overeating [3].

### Metabolic Dysfunction
Night owls face significantly elevated risks for metabolic disorders. A Korean longitudinal study of middle-aged adults found that those with late mid-sleep times (≥5:00 AM) and late bedtimes (≥2:00 AM) showed increased prevalence of pre-metabolic syndrome, with prevalence ratios of 1.61 and 1.55 respectively [4]. Mayo Clinic research emphasizes that circadian disruption represents an emerging environmental risk factor for type 2 diabetes, with individuals engaged in circadian-disrupting work conditions showing higher prevalence of diabetes, impaired glucose tolerance, and metabolic syndrome [5].

Clinical studies demonstrate that even one night of partial sleep deprivation can decrease insulin sensitivity by up to 25%, comparable to the effects of a high-fat diet. Night owls consistently show higher rates of insulin resistance, more frequent breakfast skipping, increased abdominal fat accumulation, and elevated type 2 diabetes risk [1].

## Sleep Timing vs. Total Sleep Duration: The Critical Distinction

### Timing Trumps Duration
Research increasingly demonstrates that sleep timing and consistency may be more important than total sleep duration for health outcomes. A comprehensive systematic review analyzing 41 studies with over 92,000 participants found that "sleep regularity is a stronger predictor of mortality risk than sleep duration" [6]. Later sleep timing and greater sleep variability were consistently associated with adverse health outcomes across multiple health domains.

The review revealed that social jetlag—the misalignment between biological and social time—was associated with adverse health outcomes, while weekend catch-up sleep was associated with better health outcomes. However, because most studies reported linear associations, researchers could not identify specific thresholds for what constitutes "late sleep timing" or "large sleep variability" [6].

### Circadian Alignment vs. Sleep Amount
Even when night owls obtain adequate sleep duration, the timing itself appears to matter significantly. The Stanford study found that alignment with chronotype was not the crucial factor for mental health outcomes—rather, it was specifically being up late that proved detrimental. As lead researcher Jamie Zeitzer noted, "we found that alignment with your chronotype is not crucial here, and that really it's being up late that is not good for your mental health" [3].

This suggests that the biological processes tied to circadian rhythms operate independently of sleep duration, meaning that late sleep timing can cause health problems even when total sleep time is adequate.

## Circadian Rhythm Disruption Effects on Physiological Systems

### Metabolic System Disruption
Circadian rhythm disruption profoundly affects metabolic function through multiple mechanisms. When circadian rhythms are compromised, metabolic resilience suffers, leading to reduced ATP production, increased oxidative stress, and impaired fat burning capacity. The gut microbiome, which operates on its own circadian clock, becomes imbalanced when sleep patterns are disrupted, causing bacterial imbalances that weaken the digestive lining and trigger systemic inflammation [7].

Research demonstrates that circadian disruption results in dysregulation of glucose metabolism characterized by impaired insulin secretion and insulin action. Multiple strands of evidence support a causative relationship between circadian disruption and impaired glucose homeostasis, with individuals in circadian-disrupting work conditions showing consistently higher prevalence of metabolic disorders [5].

### Cardiovascular System Impact
The cardiovascular system exhibits robust circadian oscillations that become disrupted in night owls. Blood pressure typically follows a biphasic daily pattern, characterized by nocturnal decline ("dipping") during sleep and a marked surge in the early morning upon waking. Loss of this nocturnal dipping pattern or emergence of a "reverse dipping" phenotype is strongly associated with increased risk of cardiovascular events, target organ damage, and mortality [8].

Disruption of peripheral clock gene expression, especially BMAL1, impairs normal circadian variation in vascular resistance, reduces baroreflex sensitivity, and promotes pathological blood pressure profiles. This explains why night owls show such elevated cardiovascular disease risk even when obtaining adequate sleep duration [8].

### Hormonal and Neurological Effects
Melatonin production, a critical hormone for circadian regulation and cardiovascular protection, becomes dysregulated in night owls. Melatonin acts as a potent scavenger of reactive oxygen species and stimulates expression of endogenous antioxidant enzymes. It also modulates autonomic nervous system activity by reducing sympathetic outflow, leading to decreased vascular resistance and lower resting blood pressure [8].

When natural melatonin production is delayed or reduced due to late sleep patterns, these protective effects are diminished, contributing to the elevated health risks observed in night owls.

## Potential Benefits and Neutral Outcomes for Consistent Night Owls

### Cognitive Advantages
Despite the numerous health risks, research identifies several cognitive advantages associated with night owl chronotypes. UK Biobank research on 26,820 participants aged 53-86 years found that evening types and intermediate types consistently outperformed morning types in cognitive assessments. Evening chronotypes showed the highest cognitive scores, followed by intermediate types, suggesting superior cognitive function among later chronotypes [9].

Studies indicate that night owls are often more creative and intelligent compared to early birds, with research suggesting that individuals with higher IQs tend to diverge from conventional patterns, including natural sleep-wake cycles. Night owls have the unique advantage of being able to work or learn during hours when most people are asleep, providing uninterrupted focus that can be highly beneficial in certain professions [10].

### Consistency Benefits
For night owls who maintain consistent late schedules, some health risks may be mitigated. Research shows that weekend catch-up sleep was associated with better health outcomes, and regularity in sleep patterns with consistent bedtimes and wake-up times are favorably associated with health regardless of timing [6]. This suggests that night owls who can maintain truly consistent late schedules may experience fewer adverse effects than those with variable sleep patterns.

### Professional and Creative Advantages
Night owls often excel in professions requiring creativity, problem-solving, and independent work. The quiet, distraction-free environment of nighttime hours can lead to enhanced productivity and success in fields where deep focus is essential. This professional advantage may help offset some health risks through improved socioeconomic factors and job satisfaction [10].

## Risk Mitigation Strategies for Night Owls

### Light-Based Interventions
Light therapy represents one of the most effective interventions for night owls seeking to reduce health risks. Bright light therapy is highly effective for both seasonal affective disorder and non-seasonal depression, particularly in individuals with evening chronotypes. Getting bright light exposure immediately upon waking is strongly recommended, with studies showing that exposure to 9,500 lux can shift sleep timing by 4.5 hours earlier in young adults [11].

Conversely, avoiding screens and bright light before bedtime helps maintain natural melatonin production. This dual approach of morning light exposure and evening light restriction can help partially realign circadian rhythms without completely abandoning late chronotype preferences [11].

### Lifestyle Modifications
Several lifestyle interventions can help night owls reduce health risks while maintaining later sleep schedules:

**Eating Patterns:** Time-restricted eating (TRE), which involves confining daily caloric intake to an 8-12 hour window during daylight hours, can help realign metabolic rhythms. Both animal and human studies show that TRE leads to reductions in body mass, waist circumference, blood pressure, and atherogenic lipid levels, along with improvements in insulin sensitivity and inflammatory markers [7].

**Exercise Timing:** Regular physical activity, particularly when performed in the morning or at midday, can help strengthen circadian rhythms and improve metabolic health. Morning exercise appears most beneficial for night owls seeking to mitigate health risks [11].

**Sleep Consistency:** Maintaining consistent sleep-wake schedules, even if late, is crucial. The research emphasizes that regularity may be more important than specific timing for many health outcomes [6].

### Medical Support
Night owls may benefit from medical interventions including melatonin supplementation, particularly when timed appropriately in the evening. Clinical studies demonstrate that evening melatonin administration can significantly reduce nocturnal blood pressure and improve sleep quality while better aligning with endogenous circadian rhythms [8].

Regular health screenings become particularly important for night owls given their elevated risks for cardiovascular disease, diabetes, and mental health disorders. Early detection and management of these conditions can significantly improve long-term outcomes [2].

### Harm Reduction Approaches
For night owls unable or unwilling to shift their sleep timing substantially, harm reduction strategies focus on minimizing the most dangerous aspects of late chronotypes. This includes avoiding alcohol and tobacco use, which account for much of the increased mortality risk among night owls, maintaining consistent sleep duration even with late timing, and ensuring adequate nutrition and exercise [1].

## Conclusion

The research clearly demonstrates that being a night owl carries significant long-term health implications that extend beyond simple sleep duration considerations. Even when night owls obtain adequate total sleep, late sleep timing itself appears to increase risks for cardiovascular disease, metabolic disorders, mental health problems, and early mortality.

The evidence suggests that sleep timing and circadian alignment matter independently of sleep duration, with later sleep timing consistently associated with adverse health outcomes across multiple physiological systems. However, night owls who maintain consistent schedules and implement appropriate mitigation strategies may be able to reduce some of these risks while preserving aspects of their natural chronotype preferences.

For individuals struggling with natural night owl tendencies, the research supports a balanced approach that combines gradual circadian rhythm adjustments through light therapy and lifestyle modifications with acceptance of a somewhat later sleep schedule that can be maintained consistently. The key appears to be finding a sustainable balance that respects natural chronotype preferences while minimizing the most serious health risks through evidence-based interventions.

### Sources

[1] Study Shows Night Owls Have Higher Risk of Early Death - Sleepless in Arizona: https://sleeplessinarizona.com/study-shows-night-owls-have-higher-risk-of-early-death/

[2] Staying up late is linked to poorer heart health - CNN: https://www.cnn.com/2026/01/29/health/heart-disease-risk-staying-up-late-wellness

[3] Circadian Rhythms and Metabolic Health - Energetix: https://www.energetix.com/blog/circadian-rhythms-and-metabolic-health/

[4] More than just sleeping in: a late timing of sleep is associated with health complaints - ScienceDirect: https://www.sciencedirect.com/science/article/abs/pii/S1389945718308955

[5] An emerging connection between circadian rhythm disruption and type 2 diabetes mellitus - Mayo Clinic: https://www.mayoclinic.org/medical-professionals/endocrinology/news/an-emerging-connection-between-circadian-rhythm-disruption-and-type-2-diabetes-mellitus/mac-20429399

[6] Sleep timing, sleep consistency, and health in adults: a systematic review - Canadian Science Publishing: https://cdnsciencepub.com/doi/10.1139/apnm-2020-0032

[7] Circadian Rhythms and Metabolic Health - Energetix: https://www.energetix.com/blog/circadian-rhythms-and-metabolic-health/

[8] Understanding Sleep-Wake Behavior in Late Chronotype Adolescents - PMC: https://pmc.ncbi.nlm.nih.gov/articles/PMC8963423/

[9] Sleep duration, chronotype, health and lifestyle factors affect cognition - BMJ Public Health: https://bmjpublichealth.bmj.com/content/2/1/e001000

[10] Night Owl Chronotype - Manta Sleep: https://mantasleep.com/blogs/sleep/night-owl-chronotype

[11] Understanding Sleep-Wake Behavior in Late Chronotype Adolescents - PMC: https://pmc.ncbi.nlm.nih.gov/articles/PMC8963423/


Research workflow completed!
